In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math


In [4]:
import random
import numpy as np
from numpy import ndarray

In [32]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [6]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [7]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ), (LB, LQ), (SB, MQ), (MB, SQ), (SB, SQ), (SB, MQ), (SB, MQ), (SB, SQ), (SB, LQ), (MB, LQ)
 ]

In [55]:
# Compute total number of required points across all test combinations
required_points = sum(data_size for data_size, _ in test_combinations)

# Sample from the full dataset without replacement
if required_points > total_points:
    raise ValueError(f"Need {required_points} points but only {total_points} available.")

sampled_indices = np.random.choice(total_points, size=required_points, replace=False)
combined_dataset = dataset[sampled_indices]


In [56]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


In [ ]:
print(len(queries_list))
print(len(combined_dataset))

10
2260000


**-------------Cover Tree---------------**

**Izbicki's implementation**

In [ ]:
import numpy as np
from typing import Optional, List, Tuple
from dataclasses import dataclass

@dataclass
class Node:
    point: np.ndarray
    level: int
    children: List['Node']
    _maxdist_cache: Optional[float] = None  # Cache for maxdist

    @property
    def covdist(self) -> float:
        """Coverage distance of the node."""
        return 2.0 ** self.level

    @property
    def maxdist(self) -> float:
        """Maximum distance within the subtree rooted at this node."""
        if self._maxdist_cache is None:
            if not self.children:
                self._maxdist_cache = 0
            else:
                self._maxdist_cache = max(
                    self.covdist,
                    max(child.maxdist + np.linalg.norm(self.point - child.point)
                        for child in self.children)
                )
        return self._maxdist_cache

    def update_maxdist(self):
        """Reset and recompute maxdist cache."""
        self._maxdist_cache = None
        _ = self.maxdist  # Force recomputation

class CoverTree:
    def __init__(self):
        self.root: Optional[Node] = None

    def insert(self, x: np.ndarray) -> None:
        """Insert a point into the cover tree."""
        if self.root is None:
            self.root = Node(x, 0, [])
            return

        d = np.linalg.norm(self.root.point - x)
        if d > self.root.covdist:
            # Precompute levels to expand
            levels_to_expand = int(np.ceil(np.log2(d / self.root.covdist)))
            for _ in range(levels_to_expand):
                self.root = Node(self.root.point, self.root.level + 1, [self.root])
            self.root = Node(x, self.root.level + 1, [self.root])
            return

        self._insert_recursive(self.root, x)

    def _insert_recursive(self, p: Node, x: np.ndarray) -> bool:
        """Recursive insertion into the cover tree."""
        for child in p.children:
            if np.linalg.norm(child.point - x) <= child.covdist:
                if self._insert_recursive(child, x):
                    p.update_maxdist()
                    return True

        # If x fits within the current node's coverage
        if np.linalg.norm(p.point - x) <= p.covdist:
            p.children.append(Node(x, p.level - 1, []))
            p.update_maxdist()
            return True

        return False

    def find_nearest(self, x: np.ndarray) -> Tuple[np.ndarray, float]:
        """Find the nearest neighbor to the query point."""
        if self.root is None:
            raise ValueError("Tree is empty")

        return self._find_nearest_recursive(self.root, x, self.root)

    def _find_nearest_recursive(self, p: Node, x: np.ndarray, best: Node) -> Tuple[np.ndarray, float]:
        """Recursive nearest neighbor search with pruning."""
        d_p = np.linalg.norm(p.point - x)
        d_best = np.linalg.norm(best.point - x)

        # Update the best point if the current node is closer
        if d_p < d_best:
            best = p
            d_best = d_p

        # Sort children by distance to the query point
        for child in sorted(p.children, key=lambda c: np.linalg.norm(c.point - x)):
            d_x_to_child = np.linalg.norm(child.point - x)

            # Prune subtrees based on the pruning condition
            if d_best > d_x_to_child - child.maxdist:
                child_point, child_dist = self._find_nearest_recursive(child, x, best)
                if child_dist < d_best:
                    best = Node(child_point, p.level, [])
                    d_best = child_dist

        return best.point, d_best


--------------

**----------CV Run 1----------------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 99.92it/s] 


Build Time: 372.55 sec, Search Time: 0.20 sec

1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:07<00:00, 63.64it/s]


Build Time: 445.92 sec, Search Time: 7.80 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 4.58 sec, Search Time: 0.26 sec

8: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:07<00:00, 63.38it/s]


Build Time: 4.58 sec, Search Time: 7.83 sec

9: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:08<00:00, 56.74it/s]

Build Time: 46.34 sec, Search Time: 8.75 sec

=== Summary of CV Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 372.55 sec, Search: 0.20 sec
Data: 1000000 points, Queries:  500 => Build: 445.92 sec, Search: 7.80 sec
Data:   10000 points, Queries:  100 => Build: 4.55 sec, Search: 1.72 sec
Data:  100000 points, Queries:   20 => Build: 46.60 sec, Search: 0.25 sec
Data:   10000 points, Queries:   20 => Build: 4.66 sec, Search: 0.39 sec
Data:   10000 points, Queries:  100 => Build: 4.57 sec, Search: 1.57 sec
Data:   10000 points, Queries:  100 => Build: 4.63 sec, Search: 1.82 sec
Data:   10000 points, Queries:   20 => Build: 4.58 sec, Search: 0.26 sec
Data:   10000 points, Queries:  500 => Build: 4.58 sec, Search: 7.83 sec
Data:  100000 points, Queries:  500 => Build: 46.34 sec, Search: 8.75 sec

=== Summary of CV Tree Tests ===

Total Build Time: 938.96 sec, Total Search Time: 30.59 sec
--------------------------------------------
Total Time: 969.56


**--------CV Run 2---------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 54.51it/s]


Build Time: 408.67 sec, Search Time: 0.36 sec

1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:11<00:00, 44.35it/s]


Build Time: 488.83 sec, Search Time: 11.19 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 4.88 sec, Search Time: 0.52 sec

8: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:11<00:00, 43.53it/s]


Build Time: 4.98 sec, Search Time: 11.40 sec

9: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:11<00:00, 43.06it/s]

Build Time: 51.31 sec, Search Time: 11.53 sec

=== Summary of CV Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 408.67 sec, Search: 0.36 sec
Data: 1000000 points, Queries:  500 => Build: 488.83 sec, Search: 11.19 sec
Data:   10000 points, Queries:  100 => Build: 4.93 sec, Search: 2.00 sec
Data:  100000 points, Queries:   20 => Build: 49.17 sec, Search: 0.62 sec
Data:   10000 points, Queries:   20 => Build: 4.94 sec, Search: 0.44 sec
Data:   10000 points, Queries:  100 => Build: 4.96 sec, Search: 2.38 sec
Data:   10000 points, Queries:  100 => Build: 4.99 sec, Search: 2.15 sec
Data:   10000 points, Queries:   20 => Build: 4.88 sec, Search: 0.52 sec
Data:   10000 points, Queries:  500 => Build: 4.98 sec, Search: 11.40 sec
Data:  100000 points, Queries:  500 => Build: 51.31 sec, Search: 11.53 sec

=== Summary of CV Tree Tests ===

Total Build Time: 1027.65 sec, Total Search Time: 42.59 sec
--------------------------------------------
Total Time: 1070.24


**---------CV Run 3------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 104.33it/s]


Build Time: 381.62 sec, Search Time: 0.19 sec

1: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:06<00:00, 82.99it/s]


Build Time: 458.56 sec, Search Time: 5.97 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 4.60 sec, Search Time: 0.22 sec

8: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:06<00:00, 80.07it/s]


Build Time: 4.74 sec, Search Time: 6.19 sec

9: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:06<00:00, 78.59it/s]

Build Time: 47.60 sec, Search Time: 6.31 sec

=== Summary of CV Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 381.62 sec, Search: 0.19 sec
Data: 1000000 points, Queries:  500 => Build: 458.56 sec, Search: 5.97 sec
Data:   10000 points, Queries:  100 => Build: 4.66 sec, Search: 1.23 sec
Data:  100000 points, Queries:   20 => Build: 46.73 sec, Search: 0.28 sec
Data:   10000 points, Queries:   20 => Build: 4.75 sec, Search: 0.23 sec
Data:   10000 points, Queries:  100 => Build: 4.59 sec, Search: 1.31 sec
Data:   10000 points, Queries:  100 => Build: 4.73 sec, Search: 1.18 sec
Data:   10000 points, Queries:   20 => Build: 4.60 sec, Search: 0.22 sec
Data:   10000 points, Queries:  500 => Build: 4.74 sec, Search: 6.19 sec
Data:  100000 points, Queries:  500 => Build: 47.60 sec, Search: 6.31 sec

=== Summary of CV Tree Tests ===

Total Build Time: 962.58 sec, Total Search Time: 23.10 sec
--------------------------------------------
Total Time: 985.68


**-------------CV Analysis---------------------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time: 969.56',
    'Total Time: 1070.24',
    'Total Time: 985.68'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 1008.4933 sec, Std Dev: 54.0782 sec, CV: 5.36%


-------------------------

**--------------------------VP_MWV-----------------------------**

In [ ]:
import numpy as np

class VPNode:
    def __init__(self, vp=None):
        # Vantage point
        self.vp = vp

        # Distance bounds
        self.low_near = float('inf')
        self.low_far = 0
        self.high_near = float('inf')
        self.high_far = 0

        # Child nodes
        self.lowChild = None
        self.highChild = None

        # Bucket for leaf nodes
        self.bucket = []

    def is_leaf(self):
        return self.lowChild is None and self.highChild is None

class VPTree:
    def __init__(self, bucket_size=50, distance_fn=None):
        """
        Initialize a VP-Tree

        Parameters:
        - bucket_size: Square root of maximum bucket size (b where b² is max bucket size)
        - distance_fn: Distance function to use (defaults to Euclidean)
        """
        self.bucket_size = bucket_size
        self.root = None

        # Default to Euclidean distance if none provided
        self.distance_fn = distance_fn if distance_fn is not None else self._euclidean_distance

    def _euclidean_distance(self, x, y):
        """Calculate Euclidean distance between two points"""
        x = np.asarray(x)
        y = np.asarray(y)
        diff = x - y
        return np.sqrt(np.dot(diff, diff))

    def insert(self, x):
        """Insert a new datapoint x into the VP-tree"""
        # If tree is empty, create root with this point as VP
        if self.root is None:
            self.root = VPNode(vp=x)
            return

        p = self.root

        # Traverse until we reach a leaf node
        while not p.is_leaf():
            # Calculate distance from x to current vantage point
            dist = self.distance_fn(x, p.vp)

            # Determine which child to go to
            if dist < (p.low_far + p.high_near) / 2:
                # Update distance bounds
                p.low_far = max(dist, p.low_far)
                p.low_near = min(dist, p.low_near)

                # Go to low (near) child
                p = p.lowChild
            else:
                # Update distance bounds
                p.high_far = max(dist, p.high_far)
                p.high_near = min(dist, p.high_near)

                # Go to high (far) child
                p = p.highChild

        # Add x to bucket leaf node p
        p.bucket.append(x)

        # If bucket exceeds b², split it
        if len(p.bucket) > self.bucket_size**2:
            self._split_node(p)

    def _select_vantage_point(self, points):
        """Select a vantage point from the bucket
        Can use various methods; simple random selection is shown here.
        """
        # For simplicity, using the first point as VP
        # Could implement more sophisticated selection methods
        if points:
            return points[0]
        return None

    def _split_node(self, p):
        """Split a node using the CART-inspired weighted variance method"""
        points = p.bucket.copy()

        # Select vantage point from bucket
        vp = self._select_vantage_point(points)
        points.remove(vp)  # Remove VP from points list

        # Calculate distances to vantage point
        distances = [(self.distance_fn(vp, point), point) for point in points]

        # Sort by distance
        distances.sort(key=lambda x: x[0])
        sorted_points = [point for _, point in distances]
        sorted_dists = [dist for dist, _ in distances]

        # Find optimal split using weighted variance minimization
        n = len(sorted_points)
        best_split = 0
        min_weighted_variance = float('inf')

        for s in range(1, n):
            # Calculate variances for 1:s and s:n
            if s > 0:
                var_left = np.var(sorted_dists[:s]) if s > 1 else 0
            else:
                var_left = 0

            if n-s > 0:
                var_right = np.var(sorted_dists[s:]) if n-s > 1 else 0
            else:
                var_right = 0

            # Calculate weighted variance
            weighted_var = s * var_left + (n-s) * var_right

            if weighted_var < min_weighted_variance:
                min_weighted_variance = weighted_var
                best_split = s

        # Create new children nodes
        p.lowChild = VPNode(vp=vp)
        p.highChild = VPNode(vp=self._select_vantage_point(sorted_points[best_split:]))

        # Distribute points to children based on best split
        p.lowChild.bucket = sorted_points[:best_split]
        p.highChild.bucket = sorted_points[best_split:]

        # Initialize distance bounds
        if p.lowChild.bucket:
            low_dists = sorted_dists[:best_split]
            p.low_near = min(low_dists)
            p.low_far = max(low_dists)

        if p.highChild.bucket:
            # If we have a vantage point for the high child, calculate distances to it
            high_dists = [self.distance_fn(p.highChild.vp, point) for point in p.highChild.bucket]
            p.high_near = min(high_dists) if high_dists else float('inf')
            p.high_far = max(high_dists) if high_dists else 0

        # Clear the bucket as it's no longer a leaf
        p.bucket = []

    def nearest_neighbor(self, query):
        """Optimized nearest neighbor search with batch processing for bucket points"""
        best_dist = float('inf')
        best_point = None

        # Use a stack instead of recursion to reduce overhead
        # Each entry is (node, min_possible_dist)
        stack = [(self.root, 0)]

        while stack:
            node, min_dist = stack.pop()

            # Skip this node if it cannot contain a better solution
            if min_dist >= best_dist:
                continue

            # Check vantage point distance
            if node.vp is not None:
                # Calculate distance to vantage point
                diff = np.subtract(query, node.vp)
                dist_to_vp = np.sqrt(np.dot(diff, diff))

                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.vp

            # If leaf node, check all points in bucket using batch processing
            if node.is_leaf():
                if node.bucket:
                    # Convert bucket to numpy array once
                    bucket_array = np.array(node.bucket)

                    # Vectorized distance calculation
                    query_array = np.array(query)
                    diffs = bucket_array - query_array
                    squared_dists = np.sum(diffs**2, axis=1)

                    # Find minimum distance point without unnecessary sqrt operations
                    min_idx = np.argmin(squared_dists)
                    min_squared_dist = squared_dists[min_idx]

                    # Only compute sqrt for potentially better distance
                    if min_squared_dist < best_dist**2:
                        min_dist = np.sqrt(min_squared_dist)
                        if min_dist < best_dist:
                            best_dist = min_dist
                            best_point = node.bucket[min_idx]
                continue

            # Calculate threshold
            threshold = (node.low_far + node.high_near) / 2

            # Calculate bounds for both children
            if dist_to_vp < threshold:
                # Query is in near region
                # Add far child to stack first (so near child gets processed first)
                if node.highChild:
                    far_bound = max(0, dist_to_vp - node.low_far)
                    if far_bound < best_dist:
                        stack.append((node.highChild, far_bound))

                # Add near child with zero bound (must contain query)
                if node.lowChild:
                    stack.append((node.lowChild, 0))
            else:
                # Query is in far region
                # Add near child to stack first (so far child gets processed first)
                if node.lowChild:
                    near_bound = max(0, node.high_near - dist_to_vp)
                    if near_bound < best_dist:
                        stack.append((node.lowChild, near_bound))

                # Add far child with zero bound (must contain query)
                if node.highChild:
                    stack.append((node.highChild, 0))

        return best_dist, best_point

------------

**VP_MWV (Run 1)**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 76.46it/s]


Build Time: 232.33 sec, Search Time: 0.26 sec

2: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 628.66it/s]


Build Time: 241.79 sec, Search Time: 0.79 sec

3: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 4.34 sec, Search Time: 0.03 sec

9: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 846.08it/s]


Build Time: 1.75 sec, Search Time: 0.59 sec

10: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 803.30it/s]

Build Time: 24.54 sec, Search Time: 0.62 sec

=== Summary of Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 232.33 sec, Search: 0.26 sec
Data: 1000000 points, Queries:  500 => Build: 241.79 sec, Search: 0.79 sec
Data:   10000 points, Queries:  100 => Build: 2.74 sec, Search: 0.12 sec
Data:  100000 points, Queries:   20 => Build: 25.40 sec, Search: 0.02 sec
Data:   10000 points, Queries:   20 => Build: 2.71 sec, Search: 0.03 sec
Data:   10000 points, Queries:  100 => Build: 1.41 sec, Search: 0.12 sec
Data:   10000 points, Queries:  100 => Build: 3.08 sec, Search: 0.13 sec
Data:   10000 points, Queries:   20 => Build: 4.34 sec, Search: 0.03 sec
Data:   10000 points, Queries:  500 => Build: 1.75 sec, Search: 0.59 sec
Data:  100000 points, Queries:  500 => Build: 24.54 sec, Search: 0.62 sec
--------------------------------------------

=== Summary of VP MWV Tree Tests ===

Total Build Time: 540.09 sec, Total Search Time: 2.70 sec
--------------------------------------------
T

**-----VP_MWV Run 2-------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 235.10it/s]


Build Time: 244.47 sec, Search Time: 0.08 sec

2: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 591.00it/s]


Build Time: 262.77 sec, Search Time: 0.84 sec

3: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 3.29 sec, Search Time: 0.03 sec

9: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 812.98it/s]


Build Time: 2.88 sec, Search Time: 0.61 sec

10: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 859.89it/s]

Build Time: 22.55 sec, Search Time: 0.58 sec

=== Summary of Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 244.47 sec, Search: 0.08 sec
Data: 1000000 points, Queries:  500 => Build: 262.77 sec, Search: 0.84 sec
Data:   10000 points, Queries:  100 => Build: 2.50 sec, Search: 0.15 sec
Data:  100000 points, Queries:   20 => Build: 26.61 sec, Search: 0.03 sec
Data:   10000 points, Queries:   20 => Build: 1.52 sec, Search: 0.03 sec
Data:   10000 points, Queries:  100 => Build: 1.90 sec, Search: 0.13 sec
Data:   10000 points, Queries:  100 => Build: 3.64 sec, Search: 0.15 sec
Data:   10000 points, Queries:   20 => Build: 3.29 sec, Search: 0.03 sec
Data:   10000 points, Queries:  500 => Build: 2.88 sec, Search: 0.61 sec
Data:  100000 points, Queries:  500 => Build: 22.55 sec, Search: 0.58 sec
--------------------------------------------

=== Summary of VP MWV Tree Tests ===

Total Build Time: 572.12 sec, Total Search Time: 2.62 sec
--------------------------------------------
T

**-------VP_MWV Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 203.22it/s]


Build Time: 247.24 sec, Search Time: 0.10 sec

2: === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 645.92it/s]


Build Time: 261.57 sec, Search Time: 0.77 sec

3: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 1.80 sec, Search Time: 0.03 sec

9: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 714.65it/s]


Build Time: 2.25 sec, Search Time: 0.69 sec

10: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 828.43it/s]


Build Time: 28.82 sec, Search Time: 0.60 sec

=== Summary of Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 247.24 sec, Search: 0.10 sec
Data: 1000000 points, Queries:  500 => Build: 261.57 sec, Search: 0.77 sec
Data:   10000 points, Queries:  100 => Build: 1.76 sec, Search: 0.13 sec
Data:  100000 points, Queries:   20 => Build: 25.97 sec, Search: 0.03 sec
Data:   10000 points, Queries:   20 => Build: 2.54 sec, Search: 0.03 sec
Data:   10000 points, Queries:  100 => Build: 1.80 sec, Search: 0.12 sec
Data:   10000 points, Queries:  100 => Build: 2.46 sec, Search: 0.11 sec
Data:   10000 points, Queries:   20 => Build: 1.80 sec, Search: 0.03 sec
Data:   10000 points, Queries:  500 => Build: 2.25 sec, Search: 0.69 sec
Data:  100000 points, Queries:  500 => Build: 28.82 sec, Search: 0.60 sec
--------------------------------------------

=== Summary of VP MWV Tree Tests ===

Total Build Time: 576.20 sec, Total Search Time: 2.62 sec
--------------------------------------------
T

**-----------VP MWV Analysis-----------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time: 542.79',
    'Total Time: 574.74',
    'Total Time: 578.81'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 565.4467 sec, Std Dev: 19.7265 sec, CV: 3.49%


-----------------------------------------------

**------------------Ball Tree--------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=40):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
    #*-----------------------------------------------------------------------------

    def insert(self, point):
        """
        Insert a new point into the Ball Tree.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        # Path to keep track of nodes to update
        path = []

        # Traverse the tree to find the appropriate leaf
        current = self.root
        path.append(current)

        while current.points is None:  # Continue until we reach a leaf node
            # Calculate distances to pivots
            dist_to_p1 = np.sqrt(np.sum((point - current.pivot1) ** 2))
            dist_to_p2 = np.sqrt(np.sum((point - current.pivot2) ** 2))

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid)
        # New centroid = (n * old_centroid + new_point) / (n + 1)
        current.center = (n * current.center + point) / (n + 1)

        # Update the radius if necessary
        dist_to_center = np.sqrt(np.sum((point - current.center) ** 2))
        current.radius = max(current.radius, dist_to_center)

        # Update centers and radii of ancestor nodes
        path.pop()  # Remove the leaf node as we've already updated it
        self._update_ancestors(path, point)

    #*-----------------------------------------------------------------------------
    def _update_ancestors(self, path, point):
        """
        Update the center and radius of all nodes in the path after insertion.

        Parameters:
            path (list): List of nodes from root to leaf (excluding the leaf)
            point (np.ndarray): The point that was inserted
        """
        # Process nodes from bottom to top (leaf's parent first)
        for node in reversed(path):
            # For center, we would ideally recompute based on all points
            # but as an approximation, we'll slightly shift it toward the new point
            # This way we avoid having to recursively gather all points

            # We'll apply a small weight to the new point based on estimated subtree size
            # Assume a balanced tree with approximately 2^depth nodes at the bottom
            depth = len(path) - path.index(node)
            weight = 1.0 / (2**depth)

            # Update center with minimal shift toward new point
            node.center = (1 - weight) * node.center + weight * point

            # Check if radius needs update by calculating distance from new point to center
            dist_to_center = np.sqrt(np.sum((point - node.center) ** 2))

            # Only update radius if the new point is outside the current ball
            if dist_to_center > node.radius:
                node.radius = dist_to_center

    #*-----------------------------------------------------------------------------
    def insert_opt(self, point):
        """
        Insert a new point into the Ball Tree with optimized performance.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        current = self.root
        nodes_to_update = []  # Store nodes that need radius updates

        # Traverse the tree to find the appropriate leaf
        while current.points is None:  # Continue until we reach a leaf node
            # Check if point is within current node's radius
            dist_to_center = np.linalg.norm(point - current.center)
            if dist_to_center > current.radius:
                nodes_to_update.append((current, dist_to_center))

            # Calculate distances to pivots (only once per level)
            dist_to_p1 = np.linalg.norm(point - current.pivot1)
            dist_to_p2 = np.linalg.norm(point - current.pivot2)

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid) - avoid creating temporary arrays
        current.center *= n
        current.center += point
        current.center /= (n + 1)

        # Update the radius if necessary
        dist_to_center = np.linalg.norm(point - current.center)
        if dist_to_center > current.radius:
            current.radius = dist_to_center

        # Update only radii of nodes that need updates
        for node, dist in nodes_to_update:
            node.radius = max(node.radius, dist)
#*-----------------------------------------------------------------------------

**--------Ball Tree (Run 1)----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 1147.51it/s]


Build Time: 2.56 sec, Search Time: 0.02 sec

2:  === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1037.03it/s]


Build Time: 5.71 sec, Search Time: 0.48 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 5.95 sec, Search Time: 0.02 sec

9:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 985.86it/s]


Build Time: 5.98 sec, Search Time: 0.50 sec

10:  === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 954.30it/s]

Build Time: 6.29 sec, Search Time: 0.52 sec

=== Summary of Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 2.56 sec, Search: 0.02 sec
Data: 1000000 points, Queries:  500 => Build: 5.71 sec, Search: 0.48 sec
Data:   10000 points, Queries:  100 => Build: 5.55 sec, Search: 0.10 sec
Data:  100000 points, Queries:   20 => Build: 5.81 sec, Search: 0.02 sec
Data:   10000 points, Queries:   20 => Build: 5.85 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 5.84 sec, Search: 0.10 sec
Data:   10000 points, Queries:  100 => Build: 5.90 sec, Search: 0.10 sec
Data:   10000 points, Queries:   20 => Build: 5.95 sec, Search: 0.02 sec
Data:   10000 points, Queries:  500 => Build: 5.98 sec, Search: 0.50 sec
Data:  100000 points, Queries:  500 => Build: 6.29 sec, Search: 0.52 sec
--------------------------------------------

=== Summary of Ball Tree Tests ===

Total Build Time: 55.45 sec, Total Search Time: 1.88 sec
--------------------------------------------
Total Time:

**-------Ball Run 2----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 1129.25it/s]


Build Time: 2.47 sec, Search Time: 0.02 sec

2:  === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1039.10it/s]


Build Time: 5.74 sec, Search Time: 0.48 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 5.98 sec, Search Time: 0.02 sec

9:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 956.79it/s]


Build Time: 6.04 sec, Search Time: 0.52 sec

10:  === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 939.74it/s]

Build Time: 6.41 sec, Search Time: 0.53 sec

=== Summary of Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 2.47 sec, Search: 0.02 sec
Data: 1000000 points, Queries:  500 => Build: 5.74 sec, Search: 0.48 sec
Data:   10000 points, Queries:  100 => Build: 5.70 sec, Search: 0.10 sec
Data:  100000 points, Queries:   20 => Build: 5.91 sec, Search: 0.02 sec
Data:   10000 points, Queries:   20 => Build: 5.90 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 5.91 sec, Search: 0.10 sec
Data:   10000 points, Queries:  100 => Build: 5.98 sec, Search: 0.10 sec
Data:   10000 points, Queries:   20 => Build: 5.98 sec, Search: 0.02 sec
Data:   10000 points, Queries:  500 => Build: 6.04 sec, Search: 0.52 sec
Data:  100000 points, Queries:  500 => Build: 6.41 sec, Search: 0.53 sec
--------------------------------------------

=== Summary of Ball Tree Tests ===

Total Build Time: 56.05 sec, Total Search Time: 1.90 sec
--------------------------------------------
Total Time:

**---------Ball Tree Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 1012.69it/s]


Build Time: 2.97 sec, Search Time: 0.02 sec

2:  === Testing with Data Size: 1,000,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 971.09it/s]


Build Time: 5.75 sec, Search Time: 0.51 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


...

Build Time: 6.07 sec, Search Time: 0.02 sec

9:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 965.32it/s]


Build Time: 6.06 sec, Search Time: 0.51 sec

10:  === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 992.73it/s]

Build Time: 6.53 sec, Search Time: 0.50 sec

=== Summary of Tree Tests ===
Data: 1000000 points, Queries:   20 => Build: 2.97 sec, Search: 0.02 sec
Data: 1000000 points, Queries:  500 => Build: 5.75 sec, Search: 0.51 sec
Data:   10000 points, Queries:  100 => Build: 5.63 sec, Search: 0.09 sec
Data:  100000 points, Queries:   20 => Build: 6.00 sec, Search: 0.02 sec
Data:   10000 points, Queries:   20 => Build: 5.96 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 5.97 sec, Search: 0.11 sec
Data:   10000 points, Queries:  100 => Build: 6.00 sec, Search: 0.10 sec
Data:   10000 points, Queries:   20 => Build: 6.07 sec, Search: 0.02 sec
Data:   10000 points, Queries:  500 => Build: 6.06 sec, Search: 0.51 sec
Data:  100000 points, Queries:  500 => Build: 6.53 sec, Search: 0.50 sec
--------------------------------------------

=== Summary of Ball Tree Tests ===

Total Build Time: 56.94 sec, Total Search Time: 1.91 sec
--------------------------------------------
Total Time:

**--------Ball Tree Analysis--------------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time: 57.33',
    'Total Time: 57.94',
    'Total Time: 58.85'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 58.0400 sec, Std Dev: 0.7649 sec, CV: 1.32%


-----------------------------------------------

**----------QuickSelect Algorithm-----------**

In [10]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

-----------------------------------------

**--------KD Tree Hybrid---------**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

-------------------------------------

**----------KD Tree Hybrid Sqrt Threshold Run 1-----------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 5.5561 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 28.63it/s]


Total Search Time: 0.897756 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:09<00:00, 111014.70it/s]


Insert Time: 9.0098 sec


...

Total Search Time: 31.728175 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 105229.14it/s]


Insert Time: 0.9533 sec


Querying batch 10: 100%|██████████| 500/500 [00:31<00:00, 15.90it/s]

Total Search Time: 31.928494 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.5561 sec, Total Search Time: 0.897756 sec, Total Time: 6.4538 sec
Points: 2,000,000, Insert Time: 9.0098 sec, Total Search Time: 18.958350 sec, Total Time: 27.9681 sec
Points: 2,010,000, Build Time: 12.8393 sec, Total Search Time: 6.657457 sec, Total Time: 19.4967 sec
Points: 2,110,000, Insert Time: 0.8860 sec, Total Search Time: 1.602301 sec, Total Time: 2.4883 sec
Points: 2,120,000, Insert Time: 0.0898 sec, Total Search Time: 1.672306 sec, Total Time: 1.7621 sec
Points: 2,130,000, Insert Time: 0.0868 sec, Total Search Time: 6.529848 sec, Total Time: 6.6167 sec
Points: 2,140,000, Insert Time: 0.1184 sec, Total Search Time: 6.713823 sec, Total Time: 6.8322 sec
Points: 2,150,000, Insert Time: 0.1208 sec, Total Search Time: 1.671065 sec, Total Time: 1.7918 sec
Points: 2,160,000, Insert Time: 0.0821 sec, Total Search Time: 31.728175 sec, Total Time: 31.8102 sec
Points: 2,260,000, Ins

**---------KD Tree Hybrid Sqrt Threshold Run 2--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 7.5197 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 33.18it/s]


Total Search Time: 0.803909 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:08<00:00, 111438.36it/s]


Insert Time: 8.9775 sec


...

Total Search Time: 33.833299 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 106380.79it/s]


Insert Time: 0.9426 sec


Querying batch 10: 100%|██████████| 500/500 [00:33<00:00, 14.72it/s]

Total Search Time: 34.531606 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 7.5197 sec, Total Search Time: 0.803909 sec, Total Time: 8.3236 sec
Points: 2,000,000, Insert Time: 8.9775 sec, Total Search Time: 20.281706 sec, Total Time: 29.2592 sec
Points: 2,010,000, Build Time: 15.1633 sec, Total Search Time: 7.046899 sec, Total Time: 22.2102 sec
Points: 2,110,000, Insert Time: 0.8786 sec, Total Search Time: 1.817130 sec, Total Time: 2.6958 sec
Points: 2,120,000, Insert Time: 0.1296 sec, Total Search Time: 1.850821 sec, Total Time: 1.9804 sec
Points: 2,130,000, Insert Time: 0.0838 sec, Total Search Time: 7.257634 sec, Total Time: 7.3415 sec
Points: 2,140,000, Insert Time: 0.0805 sec, Total Search Time: 7.152510 sec, Total Time: 7.2330 sec
Points: 2,150,000, Insert Time: 0.0979 sec, Total Search Time: 1.846632 sec, Total Time: 1.9446 sec
Points: 2,160,000, Insert Time: 0.0903 sec, Total Search Time: 33.833299 sec, Total Time: 33.9236 sec
Points: 2,260,000, Ins

**-------KD Tree Hybrid Sqrt Threshold Run 3-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 6.8414 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 33.55it/s]


Total Search Time: 0.794470 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:08<00:00, 112339.67it/s]


Insert Time: 8.9048 sec


...

Total Search Time: 34.015282 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 106886.06it/s]


Insert Time: 0.9383 sec


Querying batch 10: 100%|██████████| 500/500 [00:34<00:00, 14.65it/s]

Total Search Time: 34.687062 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 6.8414 sec, Total Search Time: 0.794470 sec, Total Time: 7.6358 sec
Points: 2,000,000, Insert Time: 8.9048 sec, Total Search Time: 20.190810 sec, Total Time: 29.0956 sec
Points: 2,010,000, Build Time: 14.3450 sec, Total Search Time: 7.017098 sec, Total Time: 21.3621 sec
Points: 2,110,000, Insert Time: 0.8738 sec, Total Search Time: 1.768562 sec, Total Time: 2.6424 sec
Points: 2,120,000, Insert Time: 0.0857 sec, Total Search Time: 1.868318 sec, Total Time: 1.9540 sec
Points: 2,130,000, Insert Time: 0.1175 sec, Total Search Time: 7.362015 sec, Total Time: 7.4795 sec
Points: 2,140,000, Insert Time: 0.1105 sec, Total Search Time: 7.220323 sec, Total Time: 7.3309 sec
Points: 2,150,000, Insert Time: 0.0933 sec, Total Search Time: 1.833867 sec, Total Time: 1.9272 sec
Points: 2,160,000, Insert Time: 0.0987 sec, Total Search Time: 34.015282 sec, Total Time: 34.1140 sec
Points: 2,260,000, Ins

**--------KD Tree HybridSqrt Analysis---------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 125.2885',
    'Total Time:: 115.8853',
    'Total Time:: 117.0558'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 119.4099 sec, Std Dev: 5.1246 sec, CV: 4.29%


------------------------

**------KD Tree Hybrid Log Threshold Run 1--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 5.7333 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 35.98it/s]


Total Search Time: 0.747368 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:08<00:00, 114718.12it/s]


Insert Time: 8.7203 sec


...

Total Search Time: 22.290765 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 121810.48it/s]


Insert Time: 0.8234 sec


Querying batch 10: 100%|██████████| 500/500 [00:21<00:00, 22.73it/s]

Total Search Time: 22.899116 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.7333 sec, Total Search Time: 0.747368 sec, Total Time: 6.4807 sec
Points: 2,000,000, Insert Time: 8.7203 sec, Total Search Time: 19.056385 sec, Total Time: 27.7767 sec
Points: 2,010,000, Insert Time: 0.0981 sec, Total Search Time: 4.919070 sec, Total Time: 5.0171 sec
Points: 2,110,000, Insert Time: 0.9023 sec, Total Search Time: 1.534230 sec, Total Time: 2.4365 sec
Points: 2,120,000, Insert Time: 0.0817 sec, Total Search Time: 1.629165 sec, Total Time: 1.7108 sec
Points: 2,130,000, Insert Time: 0.0944 sec, Total Search Time: 4.959484 sec, Total Time: 5.0538 sec
Points: 2,140,000, Insert Time: 0.0825 sec, Total Search Time: 4.999991 sec, Total Time: 5.0825 sec
Points: 2,150,000, Insert Time: 0.0985 sec, Total Search Time: 1.680178 sec, Total Time: 1.7786 sec
Points: 2,160,000, Insert Time: 0.1403 sec, Total Search Time: 22.290765 sec, Total Time: 22.4311 sec
Points: 2,260,000, Inse

**-------KD Hybrid Log Threshold Run 2-----------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 7.1568 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 34.40it/s]


Total Search Time: 0.774326 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:09<00:00, 110815.45it/s]


Insert Time: 9.0272 sec


...

Total Search Time: 21.526507 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 114299.98it/s]


Insert Time: 0.8769 sec


Querying batch 10: 100%|██████████| 500/500 [00:20<00:00, 23.87it/s]

Total Search Time: 22.155285 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 7.1568 sec, Total Search Time: 0.774326 sec, Total Time: 7.9312 sec
Points: 2,000,000, Insert Time: 9.0272 sec, Total Search Time: 20.619674 sec, Total Time: 29.6469 sec
Points: 2,010,000, Insert Time: 0.0940 sec, Total Search Time: 4.819573 sec, Total Time: 4.9135 sec
Points: 2,110,000, Insert Time: 0.8887 sec, Total Search Time: 1.863861 sec, Total Time: 2.7525 sec
Points: 2,120,000, Insert Time: 0.0857 sec, Total Search Time: 1.839242 sec, Total Time: 1.9250 sec
Points: 2,130,000, Insert Time: 0.1107 sec, Total Search Time: 4.952790 sec, Total Time: 5.0635 sec
Points: 2,140,000, Insert Time: 0.0910 sec, Total Search Time: 5.209828 sec, Total Time: 5.3008 sec
Points: 2,150,000, Insert Time: 0.0938 sec, Total Search Time: 1.816402 sec, Total Time: 1.9102 sec
Points: 2,160,000, Insert Time: 0.0837 sec, Total Search Time: 21.526507 sec, Total Time: 21.6102 sec
Points: 2,260,000, Inse

**------KD Hybrid Log Threshold Run 3------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 6.8377 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 34.95it/s]


Total Search Time: 0.776844 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:08<00:00, 116202.12it/s]


Insert Time: 8.6097 sec


...

Total Search Time: 21.244961 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 115496.92it/s]


Insert Time: 0.8689 sec


Querying batch 10: 100%|██████████| 500/500 [00:20<00:00, 24.64it/s]

Total Search Time: 21.428738 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 6.8377 sec, Total Search Time: 0.776844 sec, Total Time: 7.6145 sec
Points: 2,000,000, Insert Time: 8.6097 sec, Total Search Time: 20.091011 sec, Total Time: 28.7008 sec
Points: 2,010,000, Insert Time: 0.0986 sec, Total Search Time: 4.654045 sec, Total Time: 4.7526 sec
Points: 2,110,000, Insert Time: 0.9689 sec, Total Search Time: 1.831158 sec, Total Time: 2.8001 sec
Points: 2,120,000, Insert Time: 0.0966 sec, Total Search Time: 1.841295 sec, Total Time: 1.9379 sec
Points: 2,130,000, Insert Time: 0.1026 sec, Total Search Time: 4.929941 sec, Total Time: 5.0325 sec
Points: 2,140,000, Insert Time: 0.0999 sec, Total Search Time: 5.117515 sec, Total Time: 5.2174 sec
Points: 2,150,000, Insert Time: 0.0775 sec, Total Search Time: 1.817331 sec, Total Time: 1.8948 sec
Points: 2,160,000, Insert Time: 0.0846 sec, Total Search Time: 21.244961 sec, Total Time: 21.3295 sec
Points: 2,260,000, Inse

**--------KD Tree Hybrid Log Threshold Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 77.2653',
    'Total Time:: 77.1988',
    'Total Time:: 79.4331'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 77.9657 sec, Std Dev: 1.2712 sec, CV: 1.63%


-----------------------------------------------

**--------KD Hybrid Log Ratio Run 1---------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 6.0702 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 30.85it/s]


Total Search Time: 0.832601 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:08<00:00, 117445.08it/s]


Insert Time: 8.5171 sec


...

Total Search Time: 19.654937 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 110255.66it/s]


Insert Time: 0.9090 sec


Querying batch 10: 100%|██████████| 500/500 [00:22<00:00, 22.64it/s]

Total Search Time: 23.019486 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 6.0702 sec, Total Search Time: 0.832601 sec, Total Time: 6.9028 sec
Points: 2,000,000, Insert Time: 8.5171 sec, Total Search Time: 18.720599 sec, Total Time: 27.2377 sec
Points: 2,010,000, Insert Time: 0.0869 sec, Total Search Time: 5.024605 sec, Total Time: 5.1115 sec
Points: 2,110,000, Insert Time: 0.8703 sec, Total Search Time: 1.626601 sec, Total Time: 2.4969 sec
Points: 2,120,000, Insert Time: 0.0970 sec, Total Search Time: 1.677114 sec, Total Time: 1.7741 sec
Points: 2,130,000, Insert Time: 0.1155 sec, Total Search Time: 4.938012 sec, Total Time: 5.0535 sec
Points: 2,140,000, Insert Time: 0.0853 sec, Total Search Time: 4.413353 sec, Total Time: 4.4986 sec
Points: 2,150,000, Insert Time: 0.0808 sec, Total Search Time: 1.699779 sec, Total Time: 1.7806 sec
Points: 2,160,000, Insert Time: 0.0825 sec, Total Search Time: 19.654937 sec, Total Time: 19.7374 sec
Points: 2,260,000, Inse

**------KD Hybrid Log Ratio Run 2---------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 7.6470 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 33.16it/s]


Total Search Time: 0.797853 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:08<00:00, 112457.39it/s]


Insert Time: 8.8957 sec


...

Total Search Time: 21.617980 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 114541.01it/s]


Insert Time: 0.8776 sec


Querying batch 10: 100%|██████████| 500/500 [00:20<00:00, 24.03it/s]

Total Search Time: 21.999765 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 7.6470 sec, Total Search Time: 0.797853 sec, Total Time: 8.4449 sec
Points: 2,000,000, Insert Time: 8.8957 sec, Total Search Time: 20.727117 sec, Total Time: 29.6228 sec
Points: 2,010,000, Insert Time: 0.0784 sec, Total Search Time: 4.844414 sec, Total Time: 4.9228 sec
Points: 2,110,000, Insert Time: 0.9031 sec, Total Search Time: 1.888104 sec, Total Time: 2.7912 sec
Points: 2,120,000, Insert Time: 0.0968 sec, Total Search Time: 1.871675 sec, Total Time: 1.9685 sec
Points: 2,130,000, Insert Time: 0.1296 sec, Total Search Time: 5.150810 sec, Total Time: 5.2804 sec
Points: 2,140,000, Insert Time: 0.0889 sec, Total Search Time: 5.269106 sec, Total Time: 5.3580 sec
Points: 2,150,000, Insert Time: 0.0853 sec, Total Search Time: 1.822793 sec, Total Time: 1.9081 sec
Points: 2,160,000, Insert Time: 0.1123 sec, Total Search Time: 21.617980 sec, Total Time: 21.7302 sec
Points: 2,260,000, Inse

**------KD Hybrid Log Ratio Run 3-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 7.1498 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 34.47it/s]


Total Search Time: 0.773942 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:08<00:00, 111180.10it/s]


Insert Time: 8.9986 sec


...

Total Search Time: 21.248098 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 115948.03it/s]


Insert Time: 0.8657 sec


Querying batch 10: 100%|██████████| 500/500 [00:20<00:00, 24.34it/s]

Total Search Time: 21.804457 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 7.1498 sec, Total Search Time: 0.773942 sec, Total Time: 7.9237 sec
Points: 2,000,000, Insert Time: 8.9986 sec, Total Search Time: 20.333727 sec, Total Time: 29.3323 sec
Points: 2,010,000, Insert Time: 0.0799 sec, Total Search Time: 4.848555 sec, Total Time: 4.9284 sec
Points: 2,110,000, Insert Time: 0.9085 sec, Total Search Time: 1.829892 sec, Total Time: 2.7383 sec
Points: 2,120,000, Insert Time: 0.0822 sec, Total Search Time: 1.846829 sec, Total Time: 1.9291 sec
Points: 2,130,000, Insert Time: 0.0801 sec, Total Search Time: 4.956888 sec, Total Time: 5.0370 sec
Points: 2,140,000, Insert Time: 0.0833 sec, Total Search Time: 5.202768 sec, Total Time: 5.2861 sec
Points: 2,150,000, Insert Time: 0.1003 sec, Total Search Time: 1.853821 sec, Total Time: 1.9541 sec
Points: 2,160,000, Insert Time: 0.0849 sec, Total Search Time: 21.248098 sec, Total Time: 21.3330 sec
Points: 2,260,000, Inse

**------KD Tree Log Ratio Threshold Analysis-------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 74.4544',
    'Total Time:: 76.7670',
    'Total Time:: 78.4852'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 76.5689 sec, Std Dev: 2.0227 sec, CV: 2.64%


--------------------------------------------------

**----------KD Tree Epsilon Dynamic---------**

In [11]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

------------------------

**----KD  Dyn Sqrt  Run 1-----------**

In [12]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.3140 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 37.85it/s]


Total Search Time: 0.844669 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 84154.70it/s]


Insert Time: 11.8864 sec


...

Total Search Time: 24.829659 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 86944.92it/s]


Insert Time: 1.1529 sec


Querying batch 10: 100%|██████████| 500/500 [00:24<00:00, 20.72it/s]

Total Search Time: 24.853092 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.3140 sec, Total Search Time: 0.844669 sec, Total Time: 10.1587 sec
Points: 2,000,000, Insert Time: 11.8864 sec, Total Search Time: 18.162314 sec, Total Time: 30.0487 sec
Points: 2,010,000, Build Time: 19.5305 sec, Total Search Time: 5.595967 sec, Total Time: 25.1265 sec
Points: 2,110,000, Insert Time: 1.1850 sec, Total Search Time: 1.528685 sec, Total Time: 2.7137 sec
Points: 2,120,000, Insert Time: 0.1742 sec, Total Search Time: 1.518650 sec, Total Time: 1.6929 sec
Points: 2,130,000, Insert Time: 0.1212 sec, Total Search Time: 5.347162 sec, Total Time: 5.4684 sec
Points: 2,140,000, Insert Time: 0.1146 sec, Total Search Time: 5.505883 sec, Total Time: 5.6205 sec
Points: 2,150,000, Insert Time: 0.1351 sec, Total Search Time: 1.664589 sec, Total Time: 1.7996 sec
Points: 2,160,000, Insert Time: 0.1254 sec, Total Search Time: 24.829659 sec, Total Time: 24.9551 sec
Points: 2,260,000, I

**-----KD  Dyn Sqrt  Run 2-------------**

In [17]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.3214 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 50.56it/s]


Total Search Time: 0.669214 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 85838.44it/s]


Insert Time: 11.6536 sec


...

Total Search Time: 24.275906 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 87253.29it/s]


Insert Time: 1.1488 sec


Querying batch 10: 100%|██████████| 500/500 [00:21<00:00, 22.95it/s]

Total Search Time: 22.509038 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.3214 sec, Total Search Time: 0.669214 sec, Total Time: 9.9906 sec
Points: 2,000,000, Insert Time: 11.6536 sec, Total Search Time: 13.818059 sec, Total Time: 25.4717 sec
Points: 2,010,000, Build Time: 19.6300 sec, Total Search Time: 5.281052 sec, Total Time: 24.9111 sec
Points: 2,110,000, Insert Time: 1.1303 sec, Total Search Time: 1.514317 sec, Total Time: 2.6447 sec
Points: 2,120,000, Insert Time: 0.1149 sec, Total Search Time: 1.553753 sec, Total Time: 1.6687 sec
Points: 2,130,000, Insert Time: 0.1260 sec, Total Search Time: 4.868454 sec, Total Time: 4.9945 sec
Points: 2,140,000, Insert Time: 0.1200 sec, Total Search Time: 5.493499 sec, Total Time: 5.6135 sec
Points: 2,150,000, Insert Time: 0.1363 sec, Total Search Time: 1.495779 sec, Total Time: 1.6321 sec
Points: 2,160,000, Insert Time: 0.1301 sec, Total Search Time: 24.275906 sec, Total Time: 24.4060 sec
Points: 2,260,000, In

**--------KD  Dyn Sqrt  Run 3----**

In [22]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.2343 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 51.27it/s]


Total Search Time: 0.658623 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 88378.91it/s]


Insert Time: 11.3183 sec


...

Total Search Time: 24.495099 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 70138.98it/s]


Insert Time: 1.4275 sec


Querying batch 10: 100%|██████████| 500/500 [00:21<00:00, 22.84it/s]

Total Search Time: 22.573748 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.2343 sec, Total Search Time: 0.658623 sec, Total Time: 9.8929 sec
Points: 2,000,000, Insert Time: 11.3183 sec, Total Search Time: 13.651349 sec, Total Time: 24.9697 sec
Points: 2,010,000, Build Time: 19.3928 sec, Total Search Time: 5.167070 sec, Total Time: 24.5599 sec
Points: 2,110,000, Insert Time: 1.2176 sec, Total Search Time: 1.491357 sec, Total Time: 2.7090 sec
Points: 2,120,000, Insert Time: 0.1006 sec, Total Search Time: 1.605904 sec, Total Time: 1.7065 sec
Points: 2,130,000, Insert Time: 0.1192 sec, Total Search Time: 4.906759 sec, Total Time: 5.0260 sec
Points: 2,140,000, Insert Time: 0.1271 sec, Total Search Time: 5.298569 sec, Total Time: 5.4256 sec
Points: 2,150,000, Insert Time: 0.1312 sec, Total Search Time: 1.483461 sec, Total Time: 1.6147 sec
Points: 2,160,000, Insert Time: 0.1261 sec, Total Search Time: 24.495099 sec, Total Time: 24.6212 sec
Points: 2,260,000, In

**------KD Dyn Sqrt  Analysis----**

In [25]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 133.5900',
    'Total Time:: 124.9905',
    'Total Time:: 124.5268'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 127.7024 sec, Std Dev: 5.1041 sec, CV: 4.00%


---------------------------

**-------KD  Dyn Log  Run 1-----**

In [13]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.0760 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 50.38it/s]


Total Search Time: 0.715582 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 85706.14it/s]


Insert Time: 11.6699 sec


...

Total Search Time: 14.591284 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 81560.50it/s]


Insert Time: 1.2287 sec


Querying batch 10: 100%|██████████| 500/500 [00:13<00:00, 36.23it/s]

Total Search Time: 15.072141 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.0760 sec, Total Search Time: 0.715582 sec, Total Time: 9.7915 sec
Points: 2,000,000, Insert Time: 11.6699 sec, Total Search Time: 13.661627 sec, Total Time: 25.3315 sec
Points: 2,010,000, Insert Time: 0.0996 sec, Total Search Time: 3.618178 sec, Total Time: 3.7178 sec
Points: 2,110,000, Insert Time: 1.1861 sec, Total Search Time: 1.693661 sec, Total Time: 2.8798 sec
Points: 2,120,000, Insert Time: 0.1150 sec, Total Search Time: 1.708975 sec, Total Time: 1.8240 sec
Points: 2,130,000, Insert Time: 0.1150 sec, Total Search Time: 3.681397 sec, Total Time: 3.7964 sec
Points: 2,140,000, Insert Time: 0.1531 sec, Total Search Time: 3.725841 sec, Total Time: 3.8789 sec
Points: 2,150,000, Insert Time: 0.0983 sec, Total Search Time: 1.712598 sec, Total Time: 1.8109 sec
Points: 2,160,000, Insert Time: 0.1353 sec, Total Search Time: 14.591284 sec, Total Time: 14.7266 sec
Points: 2,260,000, Ins

**-----KD Dyn Log  Run 2--------**

In [18]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 8.7796 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 51.37it/s]


Total Search Time: 0.649982 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 90617.56it/s]


Insert Time: 11.0373 sec


...

Total Search Time: 14.277427 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 91440.01it/s]


Insert Time: 1.0968 sec


Querying batch 10: 100%|██████████| 500/500 [00:13<00:00, 37.00it/s]

Total Search Time: 14.750414 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 8.7796 sec, Total Search Time: 0.649982 sec, Total Time: 9.4296 sec
Points: 2,000,000, Insert Time: 11.0373 sec, Total Search Time: 13.554390 sec, Total Time: 24.5917 sec
Points: 2,010,000, Insert Time: 0.1113 sec, Total Search Time: 3.456475 sec, Total Time: 3.5678 sec
Points: 2,110,000, Insert Time: 1.1663 sec, Total Search Time: 1.749835 sec, Total Time: 2.9161 sec
Points: 2,120,000, Insert Time: 0.1259 sec, Total Search Time: 1.676911 sec, Total Time: 1.8028 sec
Points: 2,130,000, Insert Time: 0.1050 sec, Total Search Time: 3.767249 sec, Total Time: 3.8723 sec
Points: 2,140,000, Insert Time: 0.1068 sec, Total Search Time: 3.822732 sec, Total Time: 3.9296 sec
Points: 2,150,000, Insert Time: 0.1054 sec, Total Search Time: 1.600222 sec, Total Time: 1.7056 sec
Points: 2,160,000, Insert Time: 0.1051 sec, Total Search Time: 14.277427 sec, Total Time: 14.3825 sec
Points: 2,260,000, Ins

**-----KD Dyn Log  Run 3------**

In [23]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.1017 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 51.25it/s]


Total Search Time: 0.661942 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 86239.87it/s]


Insert Time: 11.5987 sec


...

Total Search Time: 14.361473 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 89340.80it/s]


Insert Time: 1.1224 sec


Querying batch 10: 100%|██████████| 500/500 [00:13<00:00, 37.32it/s]

Total Search Time: 14.689399 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.1017 sec, Total Search Time: 0.661942 sec, Total Time: 9.7636 sec
Points: 2,000,000, Insert Time: 11.5987 sec, Total Search Time: 13.453105 sec, Total Time: 25.0518 sec
Points: 2,010,000, Insert Time: 0.1220 sec, Total Search Time: 3.641246 sec, Total Time: 3.7632 sec
Points: 2,110,000, Insert Time: 1.1114 sec, Total Search Time: 1.658980 sec, Total Time: 2.7704 sec
Points: 2,120,000, Insert Time: 0.1361 sec, Total Search Time: 1.597503 sec, Total Time: 1.7336 sec
Points: 2,130,000, Insert Time: 0.1071 sec, Total Search Time: 3.778325 sec, Total Time: 3.8854 sec
Points: 2,140,000, Insert Time: 0.1228 sec, Total Search Time: 3.830128 sec, Total Time: 3.9529 sec
Points: 2,150,000, Insert Time: 0.1130 sec, Total Search Time: 1.812625 sec, Total Time: 1.9256 sec
Points: 2,160,000, Insert Time: 0.1086 sec, Total Search Time: 14.361473 sec, Total Time: 14.4701 sec
Points: 2,260,000, Ins

**------KD Tree Dynamic Log  Analysis-------**

In [26]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 84.0582',
    'Total Time:: 82.0452',
    'Total Time:: 83.1284'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 83.0773 sec, Std Dev: 1.0075 sec, CV: 1.21%


**----------------------------------**

**------KD Dyn Log Ratio Run 1--------**

In [14]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.4318 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 49.78it/s]


Total Search Time: 0.684923 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 87116.24it/s]


Insert Time: 11.4817 sec


...

Total Search Time: 14.173425 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 94263.41it/s]


Insert Time: 1.0632 sec


Querying batch 10: 100%|██████████| 500/500 [00:13<00:00, 37.67it/s]

Total Search Time: 14.447628 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.4318 sec, Total Search Time: 0.684923 sec, Total Time: 10.1168 sec
Points: 2,000,000, Insert Time: 11.4817 sec, Total Search Time: 13.855766 sec, Total Time: 25.3375 sec
Points: 2,010,000, Insert Time: 0.1025 sec, Total Search Time: 3.572806 sec, Total Time: 3.6753 sec
Points: 2,110,000, Insert Time: 1.1678 sec, Total Search Time: 1.717945 sec, Total Time: 2.8858 sec
Points: 2,120,000, Insert Time: 0.1021 sec, Total Search Time: 1.658188 sec, Total Time: 1.7603 sec
Points: 2,130,000, Insert Time: 0.1427 sec, Total Search Time: 3.633995 sec, Total Time: 3.7767 sec
Points: 2,140,000, Insert Time: 0.1317 sec, Total Search Time: 3.700455 sec, Total Time: 3.8321 sec
Points: 2,150,000, Insert Time: 0.1117 sec, Total Search Time: 1.653612 sec, Total Time: 1.7653 sec
Points: 2,160,000, Insert Time: 0.1482 sec, Total Search Time: 14.173425 sec, Total Time: 14.3216 sec
Points: 2,260,000, In

**--------KD Dyn Log Ratio Run 2------**

In [19]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.5405 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 49.35it/s]


Total Search Time: 0.682040 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 87286.96it/s]


Insert Time: 11.4585 sec


...

Total Search Time: 14.084479 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 93752.14it/s]


Insert Time: 1.0695 sec


Querying batch 10: 100%|██████████| 500/500 [00:13<00:00, 37.63it/s]

Total Search Time: 14.464508 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.5405 sec, Total Search Time: 0.682040 sec, Total Time: 10.2225 sec
Points: 2,000,000, Insert Time: 11.4585 sec, Total Search Time: 13.588681 sec, Total Time: 25.0471 sec
Points: 2,010,000, Insert Time: 0.1047 sec, Total Search Time: 3.481661 sec, Total Time: 3.5864 sec
Points: 2,110,000, Insert Time: 1.1840 sec, Total Search Time: 1.595327 sec, Total Time: 2.7793 sec
Points: 2,120,000, Insert Time: 0.1221 sec, Total Search Time: 1.602434 sec, Total Time: 1.7246 sec
Points: 2,130,000, Insert Time: 0.1019 sec, Total Search Time: 3.705371 sec, Total Time: 3.8073 sec
Points: 2,140,000, Insert Time: 0.1142 sec, Total Search Time: 3.803778 sec, Total Time: 3.9180 sec
Points: 2,150,000, Insert Time: 0.1167 sec, Total Search Time: 1.636028 sec, Total Time: 1.7527 sec
Points: 2,160,000, Insert Time: 0.1181 sec, Total Search Time: 14.084479 sec, Total Time: 14.2025 sec
Points: 2,260,000, In

**-------KD Dynamic Log Ratio Run 3----**

In [24]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding KD Tree from scratch with 1,000,000 points.
Build Time: 9.3506 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 52.04it/s]


Total Search Time: 0.661676 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:11<00:00, 88563.26it/s]


Insert Time: 11.2933 sec


...

Total Search Time: 14.206547 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 82312.75it/s]


Insert Time: 1.2188 sec


Querying batch 10: 100%|██████████| 500/500 [00:13<00:00, 37.56it/s]

Total Search Time: 14.564087 sec

=== Summary of KD Tree Scaling Test ===
Points: 1,000,000, Build Time: 9.3506 sec, Total Search Time: 0.661676 sec, Total Time: 10.0123 sec
Points: 2,000,000, Insert Time: 11.2933 sec, Total Search Time: 13.717770 sec, Total Time: 25.0111 sec
Points: 2,010,000, Insert Time: 0.1427 sec, Total Search Time: 3.612658 sec, Total Time: 3.7553 sec
Points: 2,110,000, Insert Time: 1.1618 sec, Total Search Time: 1.692710 sec, Total Time: 2.8545 sec
Points: 2,120,000, Insert Time: 0.1304 sec, Total Search Time: 1.639269 sec, Total Time: 1.7697 sec
Points: 2,130,000, Insert Time: 0.1116 sec, Total Search Time: 3.784112 sec, Total Time: 3.8957 sec
Points: 2,140,000, Insert Time: 0.1124 sec, Total Search Time: 3.805919 sec, Total Time: 3.9183 sec
Points: 2,150,000, Insert Time: 0.1003 sec, Total Search Time: 1.633625 sec, Total Time: 1.7339 sec
Points: 2,160,000, Insert Time: 0.1207 sec, Total Search Time: 14.206547 sec, Total Time: 14.3272 sec
Points: 2,260,000, In

**-------KD  Dynamic Log Ratio  Analysis----**

In [27]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 82.9821',
    'Total Time:: 82.5744',
    'Total Time:: 83.0609'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 82.8725 sec, Std Dev: 0.2611 sec, CV: 0.32%


------------------------------------------

**----------VP Hybrid---------------------**

In [28]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

------------------------

**---------VP Hybrid Sqrt  Run 1-------**

In [29]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.9043 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1938.04it/s]


Total Search Time: 0.017427 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:18<00:00, 54825.46it/s]


Insert Time: 18.2420 sec


...

Total Search Time: 0.929537 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 54512.68it/s]


Insert Time: 1.8374 sec


Querying batch 10: 100%|██████████| 500/500 [00:01<00:00, 384.28it/s]

Total Search Time: 1.447679 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.9043 sec, Total Search Time: 0.017427 sec, Total Time: 5.9217 sec
Points: 2,000,000, Insert Time: 18.2420 sec, Total Search Time: 5.660235 sec, Total Time: 23.9022 sec
Points: 2,010,000, Build Time: 12.3661 sec, Total Search Time: 0.041179 sec, Total Time: 12.4073 sec
Points: 2,110,000, Insert Time: 1.7681 sec, Total Search Time: 0.094164 sec, Total Time: 1.8622 sec
Points: 2,120,000, Insert Time: 0.1833 sec, Total Search Time: 0.095798 sec, Total Time: 0.2791 sec
Points: 2,130,000, Insert Time: 0.1798 sec, Total Search Time: 0.218706 sec, Total Time: 0.3985 sec
Points: 2,140,000, Insert Time: 0.1966 sec, Total Search Time: 0.255888 sec, Total Time: 0.4525 sec
Points: 2,150,000, Insert Time: 0.1912 sec, Total Search Time: 0.117877 sec, Total Time: 0.3091 sec
Points: 2,160,000, Insert Time: 0.1851 sec, Total Search Time: 0.929537 sec, Total Time: 1.1146 sec
Points: 2,260,000, Insert

**----------VP Hybrid Sqrt Threshold Run 2----------**

In [35]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.7608 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3438.10it/s]


Total Search Time: 0.011478 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 58702.40it/s]


Insert Time: 17.0377 sec


...

Total Search Time: 1.021479 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 57292.50it/s]


Insert Time: 1.7486 sec


Querying batch 10: 100%|██████████| 500/500 [00:01<00:00, 332.02it/s]

Total Search Time: 1.660838 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.7608 sec, Total Search Time: 0.011478 sec, Total Time: 5.7722 sec
Points: 2,000,000, Insert Time: 17.0377 sec, Total Search Time: 6.244188 sec, Total Time: 23.2819 sec
Points: 2,010,000, Build Time: 11.7497 sec, Total Search Time: 0.048820 sec, Total Time: 11.7985 sec
Points: 2,110,000, Insert Time: 1.6855 sec, Total Search Time: 0.092581 sec, Total Time: 1.7781 sec
Points: 2,120,000, Insert Time: 0.1756 sec, Total Search Time: 0.113341 sec, Total Time: 0.2889 sec
Points: 2,130,000, Insert Time: 0.1768 sec, Total Search Time: 0.228648 sec, Total Time: 0.4054 sec
Points: 2,140,000, Insert Time: 0.1826 sec, Total Search Time: 0.241899 sec, Total Time: 0.4245 sec
Points: 2,150,000, Insert Time: 0.1731 sec, Total Search Time: 0.122568 sec, Total Time: 0.2957 sec
Points: 2,160,000, Insert Time: 0.1714 sec, Total Search Time: 1.021479 sec, Total Time: 1.1929 sec
Points: 2,260,000, Insert

**-------VP Hybrid Sqrt Threshold Run 3-----**

In [40]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.7295 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2385.43it/s]


Total Search Time: 0.015552 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 56916.76it/s]


Insert Time: 17.5734 sec


...

Total Search Time: 0.892585 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 55048.84it/s]


Insert Time: 1.8200 sec


Querying batch 10: 100%|██████████| 500/500 [00:01<00:00, 402.66it/s]

Total Search Time: 1.414119 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.7295 sec, Total Search Time: 0.015552 sec, Total Time: 5.7451 sec
Points: 2,000,000, Insert Time: 17.5734 sec, Total Search Time: 5.514852 sec, Total Time: 23.0882 sec
Points: 2,010,000, Build Time: 11.8349 sec, Total Search Time: 0.054409 sec, Total Time: 11.8893 sec
Points: 2,110,000, Insert Time: 1.7661 sec, Total Search Time: 0.104242 sec, Total Time: 1.8703 sec
Points: 2,120,000, Insert Time: 0.1850 sec, Total Search Time: 0.106483 sec, Total Time: 0.2915 sec
Points: 2,130,000, Insert Time: 0.1891 sec, Total Search Time: 0.222138 sec, Total Time: 0.4113 sec
Points: 2,140,000, Insert Time: 0.1857 sec, Total Search Time: 0.226301 sec, Total Time: 0.4120 sec
Points: 2,150,000, Insert Time: 0.1847 sec, Total Search Time: 0.134704 sec, Total Time: 0.3194 sec
Points: 2,160,000, Insert Time: 0.1806 sec, Total Search Time: 0.892585 sec, Total Time: 1.0732 sec
Points: 2,260,000, Insert

**--------VP Hybrid Sqrt Threshold Analysis-----**

In [45]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 49.9323',
    'Total Time:: 48.6476',
    'Total Time:: 48.3345'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 48.9715 sec, Std Dev: 0.8467 sec, CV: 1.73%


---------------------------------------

**-----VP Hybrid Log  Run 1------**

In [30]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.8442 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1792.36it/s]


Total Search Time: 0.019032 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 55597.69it/s]


Insert Time: 17.9895 sec


...

Total Search Time: 6.358861 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 54543.12it/s]


Insert Time: 1.8370 sec


Querying batch 10: 100%|██████████| 500/500 [00:06<00:00, 77.88it/s]

Total Search Time: 7.181242 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.8442 sec, Total Search Time: 0.019032 sec, Total Time: 5.8633 sec
Points: 2,000,000, Insert Time: 17.9895 sec, Total Search Time: 5.754921 sec, Total Time: 23.7444 sec
Points: 2,010,000, Insert Time: 0.1747 sec, Total Search Time: 1.586266 sec, Total Time: 1.7610 sec
Points: 2,110,000, Insert Time: 1.7805 sec, Total Search Time: 0.975925 sec, Total Time: 2.7564 sec
Points: 2,120,000, Insert Time: 0.1755 sec, Total Search Time: 0.921252 sec, Total Time: 1.0968 sec
Points: 2,130,000, Insert Time: 0.1859 sec, Total Search Time: 1.838016 sec, Total Time: 2.0239 sec
Points: 2,140,000, Insert Time: 0.1921 sec, Total Search Time: 1.902552 sec, Total Time: 2.0947 sec
Points: 2,150,000, Insert Time: 0.1978 sec, Total Search Time: 0.954482 sec, Total Time: 1.1522 sec
Points: 2,160,000, Insert Time: 0.1881 sec, Total Search Time: 6.358861 sec, Total Time: 6.5469 sec
Points: 2,260,000, Insert 

**-------VP Hybrid Log Threshold Run 2-----**

In [36]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.4134 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3167.19it/s]


Total Search Time: 0.010505 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 58212.15it/s]


Insert Time: 17.1816 sec


...

Total Search Time: 7.153803 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 58783.04it/s]


Insert Time: 1.7040 sec


Querying batch 10: 100%|██████████| 500/500 [00:07<00:00, 67.79it/s]

Total Search Time: 8.087886 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.4134 sec, Total Search Time: 0.010505 sec, Total Time: 5.4239 sec
Points: 2,000,000, Insert Time: 17.1816 sec, Total Search Time: 6.501667 sec, Total Time: 23.6832 sec
Points: 2,010,000, Insert Time: 0.1752 sec, Total Search Time: 1.951343 sec, Total Time: 2.1265 sec
Points: 2,110,000, Insert Time: 1.8109 sec, Total Search Time: 0.905764 sec, Total Time: 2.7166 sec
Points: 2,120,000, Insert Time: 0.1715 sec, Total Search Time: 0.942668 sec, Total Time: 1.1142 sec
Points: 2,130,000, Insert Time: 0.1740 sec, Total Search Time: 1.874754 sec, Total Time: 2.0487 sec
Points: 2,140,000, Insert Time: 0.1713 sec, Total Search Time: 1.950591 sec, Total Time: 2.1219 sec
Points: 2,150,000, Insert Time: 0.1717 sec, Total Search Time: 0.891247 sec, Total Time: 1.0629 sec
Points: 2,160,000, Insert Time: 0.1754 sec, Total Search Time: 7.153803 sec, Total Time: 7.3292 sec
Points: 2,260,000, Insert 

**-----VP Hybrid Log Threshold Run 3--------**

In [41]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.8809 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2498.69it/s]


Total Search Time: 0.014413 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 56691.01it/s]


Insert Time: 17.6421 sec


...

Total Search Time: 6.339948 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 56416.52it/s]


Insert Time: 1.7746 sec


Querying batch 10: 100%|██████████| 500/500 [00:06<00:00, 81.13it/s]

Total Search Time: 6.988078 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.8809 sec, Total Search Time: 0.014413 sec, Total Time: 5.8954 sec
Points: 2,000,000, Insert Time: 17.6421 sec, Total Search Time: 5.401281 sec, Total Time: 23.0434 sec
Points: 2,010,000, Insert Time: 0.1871 sec, Total Search Time: 1.615853 sec, Total Time: 1.8029 sec
Points: 2,110,000, Insert Time: 1.8025 sec, Total Search Time: 1.012175 sec, Total Time: 2.8146 sec
Points: 2,120,000, Insert Time: 0.1770 sec, Total Search Time: 0.922397 sec, Total Time: 1.0994 sec
Points: 2,130,000, Insert Time: 0.1765 sec, Total Search Time: 1.823989 sec, Total Time: 2.0005 sec
Points: 2,140,000, Insert Time: 0.1903 sec, Total Search Time: 1.782025 sec, Total Time: 1.9723 sec
Points: 2,150,000, Insert Time: 0.1774 sec, Total Search Time: 1.027563 sec, Total Time: 1.2050 sec
Points: 2,160,000, Insert Time: 0.1836 sec, Total Search Time: 6.339948 sec, Total Time: 6.5235 sec
Points: 2,260,000, Insert 

**--------VP Tree Hybrid Log Threshold Analysis-----**

In [44]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 56.0578',
    'Total Time:: 57.4191',
    'Total Time:: 55.1198 '
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 56.1989 sec, Std Dev: 1.1561 sec, CV: 2.06%


---------------------------

**-----VP Tree Hybrid Log Ratio Run 1-------**

In [31]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 6.2738 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1776.38it/s]


Total Search Time: 0.018005 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 56445.14it/s]


Insert Time: 17.7198 sec


...

Total Search Time: 6.274515 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 58140.46it/s]


Insert Time: 1.7237 sec


Querying batch 10: 100%|██████████| 500/500 [00:06<00:00, 82.60it/s]

Total Search Time: 6.795851 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 6.2738 sec, Total Search Time: 0.018005 sec, Total Time: 6.2918 sec
Points: 2,000,000, Insert Time: 17.7198 sec, Total Search Time: 5.689685 sec, Total Time: 23.4095 sec
Points: 2,010,000, Insert Time: 0.1879 sec, Total Search Time: 1.614518 sec, Total Time: 1.8024 sec
Points: 2,110,000, Insert Time: 1.7654 sec, Total Search Time: 0.922325 sec, Total Time: 2.6877 sec
Points: 2,120,000, Insert Time: 0.1976 sec, Total Search Time: 0.942707 sec, Total Time: 1.1403 sec
Points: 2,130,000, Insert Time: 0.1739 sec, Total Search Time: 1.772487 sec, Total Time: 1.9464 sec
Points: 2,140,000, Insert Time: 0.1712 sec, Total Search Time: 1.874423 sec, Total Time: 2.0456 sec
Points: 2,150,000, Insert Time: 0.1764 sec, Total Search Time: 0.929913 sec, Total Time: 1.1063 sec
Points: 2,160,000, Insert Time: 0.1836 sec, Total Search Time: 6.274515 sec, Total Time: 6.4581 sec
Points: 2,260,000, Insert 

**------VP Hybrid Log Ratio Run 2---------**

In [37]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.6884 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3422.25it/s]


Total Search Time: 0.010609 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 57508.92it/s]


Insert Time: 17.3916 sec


...

Total Search Time: 7.021354 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 58538.04it/s]


Insert Time: 1.7109 sec


Querying batch 10: 100%|██████████| 500/500 [00:07<00:00, 69.19it/s]

Total Search Time: 7.930911 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.6884 sec, Total Search Time: 0.010609 sec, Total Time: 5.6990 sec
Points: 2,000,000, Insert Time: 17.3916 sec, Total Search Time: 6.304139 sec, Total Time: 23.6958 sec
Points: 2,010,000, Insert Time: 0.1752 sec, Total Search Time: 1.890704 sec, Total Time: 2.0659 sec
Points: 2,110,000, Insert Time: 1.7998 sec, Total Search Time: 0.877204 sec, Total Time: 2.6770 sec
Points: 2,120,000, Insert Time: 0.1698 sec, Total Search Time: 0.929452 sec, Total Time: 1.0992 sec
Points: 2,130,000, Insert Time: 0.1720 sec, Total Search Time: 1.930838 sec, Total Time: 2.1029 sec
Points: 2,140,000, Insert Time: 0.1766 sec, Total Search Time: 2.027174 sec, Total Time: 2.2038 sec
Points: 2,150,000, Insert Time: 0.1712 sec, Total Search Time: 0.892467 sec, Total Time: 1.0637 sec
Points: 2,160,000, Insert Time: 0.1740 sec, Total Search Time: 7.021354 sec, Total Time: 7.1953 sec
Points: 2,260,000, Insert 

**------VP Hybrid Log Ratio Run 3----------**

In [42]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 5.6735 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2200.87it/s]


Total Search Time: 0.016985 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:17<00:00, 56453.32it/s]


Insert Time: 17.7166 sec


...

Total Search Time: 6.476845 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 56737.00it/s]


Insert Time: 1.7657 sec


Querying batch 10: 100%|██████████| 500/500 [00:06<00:00, 81.29it/s]

Total Search Time: 6.927761 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 5.6735 sec, Total Search Time: 0.016985 sec, Total Time: 5.6905 sec
Points: 2,000,000, Insert Time: 17.7166 sec, Total Search Time: 5.465599 sec, Total Time: 23.1822 sec
Points: 2,010,000, Insert Time: 0.1792 sec, Total Search Time: 1.662315 sec, Total Time: 1.8416 sec
Points: 2,110,000, Insert Time: 1.8359 sec, Total Search Time: 1.002349 sec, Total Time: 2.8382 sec
Points: 2,120,000, Insert Time: 0.1805 sec, Total Search Time: 0.937132 sec, Total Time: 1.1176 sec
Points: 2,130,000, Insert Time: 0.1811 sec, Total Search Time: 1.814948 sec, Total Time: 1.9960 sec
Points: 2,140,000, Insert Time: 0.1802 sec, Total Search Time: 1.787049 sec, Total Time: 1.9672 sec
Points: 2,150,000, Insert Time: 0.1757 sec, Total Search Time: 1.006203 sec, Total Time: 1.1819 sec
Points: 2,160,000, Insert Time: 0.1765 sec, Total Search Time: 6.476845 sec, Total Time: 6.6533 sec
Points: 2,260,000, Insert 

**----------VP Tree Hybrid Log Ratio Threshold Analysis-----**

In [43]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 55.4076',
    'Total Time:: 57.4444',
    'Total Time:: 55.1619'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 56.0046 sec, Std Dev: 1.2529 sec, CV: 2.24%


----------------------------------

---------------------------------------------

**-------VP Tree Dynamic-------**

In [46]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

-----------------------

**-----VP Dynamic Sqrt Threshold Run 1---------**

In [47]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.6393 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3685.19it/s]


Total Search Time: 0.009018 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:14<00:00, 69862.64it/s]


Insert Time: 14.3165 sec


...

Total Search Time: 1.220240 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 70911.79it/s]


Insert Time: 1.4130 sec


Querying batch 10: 100%|██████████| 500/500 [00:01<00:00, 286.38it/s]

Total Search Time: 1.902564 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.6393 sec, Total Search Time: 0.009018 sec, Total Time: 4.6483 sec
Points: 2,000,000, Insert Time: 14.3165 sec, Total Search Time: 7.423702 sec, Total Time: 21.7402 sec
Points: 2,010,000, Build Time: 9.4864 sec, Total Search Time: 0.037249 sec, Total Time: 9.5237 sec
Points: 2,110,000, Insert Time: 1.4020 sec, Total Search Time: 0.108414 sec, Total Time: 1.5104 sec
Points: 2,120,000, Insert Time: 0.1376 sec, Total Search Time: 0.111630 sec, Total Time: 0.2492 sec
Points: 2,130,000, Insert Time: 0.1607 sec, Total Search Time: 0.251475 sec, Total Time: 0.4122 sec
Points: 2,140,000, Insert Time: 0.1362 sec, Total Search Time: 0.296493 sec, Total Time: 0.4327 sec
Points: 2,150,000, Insert Time: 0.1372 sec, Total Search Time: 0.146148 sec, Total Time: 0.2833 sec
Points: 2,160,000, Insert Time: 0.1355 sec, Total Search Time: 1.220240 sec, Total Time: 1.3558 sec
Points: 2,260,000, Insert T

**--------VP Dyn Sqrt Threshold Run 2------**

In [52]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.9970 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 4373.85it/s]


Total Search Time: 0.009705 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:14<00:00, 70687.75it/s]


Insert Time: 14.1501 sec


...

Total Search Time: 1.008799 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 72795.05it/s]


Insert Time: 1.3771 sec


Querying batch 10: 100%|██████████| 500/500 [00:01<00:00, 336.38it/s]

Total Search Time: 1.662604 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.9970 sec, Total Search Time: 0.009705 sec, Total Time: 5.0067 sec
Points: 2,000,000, Insert Time: 14.1501 sec, Total Search Time: 6.798311 sec, Total Time: 20.9484 sec
Points: 2,010,000, Build Time: 9.8154 sec, Total Search Time: 0.012191 sec, Total Time: 9.8275 sec
Points: 2,110,000, Insert Time: 1.3732 sec, Total Search Time: 0.078295 sec, Total Time: 1.4514 sec
Points: 2,120,000, Insert Time: 0.1551 sec, Total Search Time: 0.082047 sec, Total Time: 0.2372 sec
Points: 2,130,000, Insert Time: 0.1573 sec, Total Search Time: 0.219128 sec, Total Time: 0.3765 sec
Points: 2,140,000, Insert Time: 0.1560 sec, Total Search Time: 0.245058 sec, Total Time: 0.4010 sec
Points: 2,150,000, Insert Time: 0.1567 sec, Total Search Time: 0.116072 sec, Total Time: 0.2728 sec
Points: 2,160,000, Insert Time: 0.1701 sec, Total Search Time: 1.008799 sec, Total Time: 1.1789 sec
Points: 2,260,000, Insert T

**-------------VP Dyn Sqrt Threshold Run 3-------**

In [57]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.3534 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 10959.77it/s]


Total Search Time: 0.005157 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:13<00:00, 71896.36it/s]


Insert Time: 13.9122 sec


...

Total Search Time: 1.161935 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 70308.64it/s]


Insert Time: 1.4244 sec


Querying batch 10: 100%|██████████| 500/500 [00:01<00:00, 284.96it/s]

Total Search Time: 1.918061 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.3534 sec, Total Search Time: 0.005157 sec, Total Time: 4.3585 sec
Points: 2,000,000, Insert Time: 13.9122 sec, Total Search Time: 8.224744 sec, Total Time: 22.1369 sec
Points: 2,010,000, Build Time: 8.9093 sec, Total Search Time: 0.010944 sec, Total Time: 8.9203 sec
Points: 2,110,000, Insert Time: 1.3713 sec, Total Search Time: 0.096137 sec, Total Time: 1.4675 sec
Points: 2,120,000, Insert Time: 0.1629 sec, Total Search Time: 0.113529 sec, Total Time: 0.2764 sec
Points: 2,130,000, Insert Time: 0.1601 sec, Total Search Time: 0.239181 sec, Total Time: 0.3993 sec
Points: 2,140,000, Insert Time: 0.1504 sec, Total Search Time: 0.260260 sec, Total Time: 0.4107 sec
Points: 2,150,000, Insert Time: 0.1381 sec, Total Search Time: 0.135953 sec, Total Time: 0.2740 sec
Points: 2,160,000, Insert Time: 0.1361 sec, Total Search Time: 1.161935 sec, Total Time: 1.2980 sec
Points: 2,260,000, Insert T

**-------VP Dyn Sqrt Threshold Analysis----------**

In [62]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 43.4713',
    'Total Time:: 42.7401',
    'Total Time:: 42.8842'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 43.0319 sec, Std Dev: 0.3873 sec, CV: 0.90%


-----------------------

**-----VP Dynamic Log Threshold Run 1------**

In [48]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.4039 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 4327.37it/s]


Total Search Time: 0.007832 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:14<00:00, 70235.82it/s]


Insert Time: 14.2403 sec


...

Total Search Time: 8.936689 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 74095.55it/s]


Insert Time: 1.3530 sec


Querying batch 10: 100%|██████████| 500/500 [00:08<00:00, 56.29it/s]

Total Search Time: 9.729628 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.4039 sec, Total Search Time: 0.007832 sec, Total Time: 4.4117 sec
Points: 2,000,000, Insert Time: 14.2403 sec, Total Search Time: 7.429619 sec, Total Time: 21.6700 sec
Points: 2,010,000, Insert Time: 0.1489 sec, Total Search Time: 2.051082 sec, Total Time: 2.2000 sec
Points: 2,110,000, Insert Time: 1.3732 sec, Total Search Time: 1.036519 sec, Total Time: 2.4097 sec
Points: 2,120,000, Insert Time: 0.1394 sec, Total Search Time: 1.028515 sec, Total Time: 1.1679 sec
Points: 2,130,000, Insert Time: 0.1377 sec, Total Search Time: 2.284154 sec, Total Time: 2.4218 sec
Points: 2,140,000, Insert Time: 0.1397 sec, Total Search Time: 2.399704 sec, Total Time: 2.5394 sec
Points: 2,150,000, Insert Time: 0.1502 sec, Total Search Time: 1.096299 sec, Total Time: 1.2465 sec
Points: 2,160,000, Insert Time: 0.1355 sec, Total Search Time: 8.936689 sec, Total Time: 9.0722 sec
Points: 2,260,000, Insert 

**--------VP Dynamic Log Threshold Run 2--------**

In [53]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.5269 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 6313.87it/s]


Total Search Time: 0.006887 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:13<00:00, 71877.58it/s]


Insert Time: 13.9158 sec


...

Total Search Time: 7.970822 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 67995.67it/s]


Insert Time: 1.4740 sec


Querying batch 10: 100%|██████████| 500/500 [00:07<00:00, 63.14it/s]

Total Search Time: 8.798330 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.5269 sec, Total Search Time: 0.006887 sec, Total Time: 4.5338 sec
Points: 2,000,000, Insert Time: 13.9158 sec, Total Search Time: 6.758004 sec, Total Time: 20.6739 sec
Points: 2,010,000, Insert Time: 0.1534 sec, Total Search Time: 1.965959 sec, Total Time: 2.1193 sec
Points: 2,110,000, Insert Time: 1.4775 sec, Total Search Time: 1.048205 sec, Total Time: 2.5257 sec
Points: 2,120,000, Insert Time: 0.1523 sec, Total Search Time: 1.016647 sec, Total Time: 1.1689 sec
Points: 2,130,000, Insert Time: 0.1531 sec, Total Search Time: 2.138984 sec, Total Time: 2.2921 sec
Points: 2,140,000, Insert Time: 0.1500 sec, Total Search Time: 2.172067 sec, Total Time: 2.3221 sec
Points: 2,150,000, Insert Time: 0.1519 sec, Total Search Time: 1.042051 sec, Total Time: 1.1939 sec
Points: 2,160,000, Insert Time: 0.1582 sec, Total Search Time: 7.970822 sec, Total Time: 8.1290 sec
Points: 2,260,000, Insert 

**------VP Dynamic Log Threshold Run 3----------**

In [58]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.1866 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 9669.87it/s]


Total Search Time: 0.004920 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:14<00:00, 71258.71it/s]


Insert Time: 14.0360 sec


...

Total Search Time: 9.463810 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 77196.89it/s]


Insert Time: 1.2985 sec


Querying batch 10: 100%|██████████| 500/500 [00:09<00:00, 52.55it/s]

Total Search Time: 10.326854 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.1866 sec, Total Search Time: 0.004920 sec, Total Time: 4.1915 sec
Points: 2,000,000, Insert Time: 14.0360 sec, Total Search Time: 8.105232 sec, Total Time: 22.1413 sec
Points: 2,010,000, Insert Time: 0.1622 sec, Total Search Time: 2.156399 sec, Total Time: 2.3186 sec
Points: 2,110,000, Insert Time: 1.2934 sec, Total Search Time: 1.028093 sec, Total Time: 2.3215 sec
Points: 2,120,000, Insert Time: 0.1329 sec, Total Search Time: 1.041832 sec, Total Time: 1.1748 sec
Points: 2,130,000, Insert Time: 0.1290 sec, Total Search Time: 2.402170 sec, Total Time: 2.5311 sec
Points: 2,140,000, Insert Time: 0.1377 sec, Total Search Time: 2.378312 sec, Total Time: 2.5160 sec
Points: 2,150,000, Insert Time: 0.1331 sec, Total Search Time: 1.078460 sec, Total Time: 1.2115 sec
Points: 2,160,000, Insert Time: 0.1413 sec, Total Search Time: 9.463810 sec, Total Time: 9.6051 sec
Points: 2,260,000, Insert

**------VP Tree Dynamic Log Threshold Analysis------**

In [61]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 58.2218',
    'Total Time:: 55.2310',
    'Total Time:: 59.6369'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.6966 sec, Std Dev: 2.2494 sec, CV: 3.90%


------------------------------

**-----VP Dynamic Log Ratio Run 1-----**

In [49]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.6495 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 4526.06it/s]


Total Search Time: 0.007712 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:14<00:00, 70698.47it/s]


Insert Time: 14.1466 sec


...

Total Search Time: 8.785690 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 75985.50it/s]


Insert Time: 1.3191 sec


Querying batch 10: 100%|██████████| 500/500 [00:09<00:00, 55.40it/s]

Total Search Time: 9.852250 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.6495 sec, Total Search Time: 0.007712 sec, Total Time: 4.6572 sec
Points: 2,000,000, Insert Time: 14.1466 sec, Total Search Time: 7.402778 sec, Total Time: 21.5494 sec
Points: 2,010,000, Insert Time: 0.1623 sec, Total Search Time: 2.032316 sec, Total Time: 2.1946 sec
Points: 2,110,000, Insert Time: 1.3695 sec, Total Search Time: 1.071528 sec, Total Time: 2.4410 sec
Points: 2,120,000, Insert Time: 0.1401 sec, Total Search Time: 1.040327 sec, Total Time: 1.1805 sec
Points: 2,130,000, Insert Time: 0.1379 sec, Total Search Time: 2.266769 sec, Total Time: 2.4046 sec
Points: 2,140,000, Insert Time: 0.1363 sec, Total Search Time: 2.405742 sec, Total Time: 2.5421 sec
Points: 2,150,000, Insert Time: 0.1363 sec, Total Search Time: 1.093714 sec, Total Time: 1.2300 sec
Points: 2,160,000, Insert Time: 0.1386 sec, Total Search Time: 8.785690 sec, Total Time: 8.9242 sec
Points: 2,260,000, Insert 

**--------VP Dyn Log Ratio Run 2-------**

In [54]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.9122 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 6740.00it/s]


Total Search Time: 0.006614 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:13<00:00, 71466.59it/s]


Insert Time: 13.9952 sec


...

Total Search Time: 7.817092 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 66656.94it/s]


Insert Time: 1.5022 sec


Querying batch 10: 100%|██████████| 500/500 [00:07<00:00, 62.62it/s]

Total Search Time: 8.833339 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.9122 sec, Total Search Time: 0.006614 sec, Total Time: 4.9188 sec
Points: 2,000,000, Insert Time: 13.9952 sec, Total Search Time: 6.780019 sec, Total Time: 20.7752 sec
Points: 2,010,000, Insert Time: 0.1523 sec, Total Search Time: 1.891130 sec, Total Time: 2.0434 sec
Points: 2,110,000, Insert Time: 1.4651 sec, Total Search Time: 1.027368 sec, Total Time: 2.4925 sec
Points: 2,120,000, Insert Time: 0.1489 sec, Total Search Time: 1.017850 sec, Total Time: 1.1668 sec
Points: 2,130,000, Insert Time: 0.1546 sec, Total Search Time: 2.145746 sec, Total Time: 2.3004 sec
Points: 2,140,000, Insert Time: 0.1500 sec, Total Search Time: 2.139616 sec, Total Time: 2.2896 sec
Points: 2,150,000, Insert Time: 0.1531 sec, Total Search Time: 1.026132 sec, Total Time: 1.1793 sec
Points: 2,160,000, Insert Time: 0.1503 sec, Total Search Time: 7.817092 sec, Total Time: 7.9674 sec
Points: 2,260,000, Insert 

**-------VP Dyn Log Ratio Run 3--------**

In [59]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 1,000,000 points
Rebuilding VP Tree from scratch with 1,000,000 points.
Build Time: 4.4321 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 10276.38it/s]


Total Search Time: 0.004280 sec

Batch 2: Adding 1,000,000 points


Inserting 1,000,000 points: 100%|██████████| 1000000/1000000 [00:13<00:00, 71965.38it/s]


Insert Time: 13.8987 sec


...

Total Search Time: 9.338722 sec

Batch 10: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 78193.19it/s]


Insert Time: 1.2808 sec


Querying batch 10: 100%|██████████| 500/500 [00:09<00:00, 54.12it/s]

Total Search Time: 10.024431 sec

=== Summary of VP Tree Scaling Test ===
Points: 1,000,000, Build Time: 4.4321 sec, Total Search Time: 0.004280 sec, Total Time: 4.4364 sec
Points: 2,000,000, Insert Time: 13.8987 sec, Total Search Time: 8.090373 sec, Total Time: 21.9891 sec
Points: 2,010,000, Insert Time: 0.1597 sec, Total Search Time: 2.146625 sec, Total Time: 2.3064 sec
Points: 2,110,000, Insert Time: 1.3015 sec, Total Search Time: 1.019810 sec, Total Time: 2.3213 sec
Points: 2,120,000, Insert Time: 0.1333 sec, Total Search Time: 1.035953 sec, Total Time: 1.1693 sec
Points: 2,130,000, Insert Time: 0.1354 sec, Total Search Time: 2.421563 sec, Total Time: 2.5569 sec
Points: 2,140,000, Insert Time: 0.1419 sec, Total Search Time: 2.323284 sec, Total Time: 2.4652 sec
Points: 2,150,000, Insert Time: 0.1317 sec, Total Search Time: 1.075960 sec, Total Time: 1.2077 sec
Points: 2,160,000, Insert Time: 0.1335 sec, Total Search Time: 9.338722 sec, Total Time: 9.4723 sec
Points: 2,260,000, Insert

**-------VP Tree Dynamic Log Ratio Threshold Analysis-----**

In [60]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 58.2950',
    'Total Time:: 59.2297',
    'Total Time:: 55.4688'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.6645 sec, Std Dev: 1.9581 sec, CV: 3.40%


------------------------------------------------